### Modelagem SER

1. Sinal de saída da híbrida / Sinal de entrada do SER 
$$
\begin{aligned}
E_{1}(t)    &= I(t) + jQ(t) + A \\
E_{2}(t)    &= I(t) + jQ(t) + Ae^{j\frac{\pi}{2}}
\end{aligned}
$$

2. Sinal de saída do SER 
$$
\begin{aligned}
R_{1}(t)    &= A^{2} + I(t)^{2} + Q(t)^{2} + 2AI(t) \\
R_{2}(t)    &= A^{2} + I(t)^{2} + Q(t)^{2} + 2AQ(t)
\end{aligned}
$$

3. Modelo ideal (ideal square-law detectors)

\begin{aligned}
R_{1}(t)    &= \left((I(t) + A + jQ(t)    \right) \left(I(t) + A - jQ(t)\right) \\
            &= (I(t) + A)^{2} - jQ(t)(I(t) + A) + jQ(t)(I(t) + A) + Q(t)^{2}\\
            &= A^{2} + I(t)^{2} + Q(t)^{2} + 2AI(t) \\
\end{aligned}
$$

\begin{aligned}
R_{2}(t)    &= \left((I(t) + j(Q(t) + A)\right) \left((I(t) - j(Q(t) + A)\right) \\
            &= I(t)^{2} -jI(Q(t) + A) + jI(Q(t) + A) + Q^{2} + 2AQ(t) + A^{2} \\
            &= A^{2} + I(t)^{2} + Q(t)^{2} + 2AQ(t)
\end{aligned}
$$

* $2AI(t) \ e \ 2AQ(t)$: termos de detecção úteis
* $I(t)^{2} + Q(t)^{2}$: signal-signal beat interference (SSBI)

In [20]:
# Import

import sympy as sp
from IPython.display import Math, display
from sympy import Matrix, zeros

π = sp.pi
j = sp.I
t = sp.symbols('t', real=True)

# função para print de expressões simbólicas
def symdisp(expr, var, unit=' '):
    '''
    Latex style display of sympy expressions
    
    :param expr: expression in latex [string]
    :param var: sympy variable, function, expression.
    :param unit: string indicating unit of var [string]
    '''
    display(Math(expr+sp.latex(var)+'\;'+unit))
    
    
def photodiode(Ei):
    return sp.Abs(Ei)**2

def bpd(E_pos, E_neg):
    return photodiode(E_pos) - photodiode(E_neg)

In [21]:
# Sinal referência (artigo)
E_s  = sp.sqrt(2) * ( sp.Function('I', real=True)(t) + j * sp.Function('Q', real=True)(t))
E_lo = sp.sqrt(2) * ( sp.Function('A', real=True)(t) )

# Topologia do Artigo -> 2 splitter, 1 defasador, 2 acopladores de 90°
Ei = Matrix([[ E_s ], 
             [ 0 ], 
             [ 0 ], 
             [ E_lo ]])

T1 = Matrix([[1, j, 0, 0],
             [j, 1, 0, 0],
             [0, 0, 1, j],
             [0, 0, j, 1]]) * (1/sp.sqrt(2)) * (sp.sqrt(2)) 

T2 = Matrix([[1, 0, 0, 0],
             [0, 0, 1, 0],
             [0, 1, 0, 0],
             [0, 0, 0, j]])

T3 = Matrix([[1, j, 0, 0],
             [j, 1, 0, 0],
             [0, 0, 1, j],
             [0, 0, j, 1]]) * (1/sp.sqrt(2))

print('Sinais de entrada da híbrida óptica:')
symdisp('E_i = ', Ei)

print('Função de transferência da híbrida óptica:')
symdisp('T = ', T3 * T2 * T1)

print('Modelagem:')
symdisp('T1 * E_i = ', T1 * Ei)
symdisp('T2 * T1 * E_i = ', T2 * T1 * Ei )
symdisp('T3 * T2 * T1 * E_i = ', T3 * T2 * T1 * Ei )

print('Sinais de saída da híbrida óptica / Sinais de entrada dos fotodiodos:')
Eo = T3 * T2 * T1 * Ei
symdisp('E_o = ', Eo)

#BPD
print('Saída com topologia balanceada:')
sI = bpd(Eo[1], Eo[0])
sQ = bpd(Eo[2], Eo[3])

symdisp('BPD | s_I = i_1 - i_2 = ', sp.re(sI).simplify().expand() )
symdisp('BPD | s_Q = i_4 - i_3 = ', sp.re(sQ).simplify().expand() )

#SER
print('Saída com topologia single-ended:')
sI = bpd(Eo[1], 0)
sQ = bpd(Eo[2], 0)

symdisp('SER | s_I = i_1 = ', sp.re(sI).simplify().expand() )
symdisp('SER | s_Q = i_3 = ', sp.re(sQ).simplify().expand() )

Sinais de entrada da híbrida óptica:


<IPython.core.display.Math object>

Função de transferência da híbrida óptica:


<IPython.core.display.Math object>

Modelagem:


...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Saída com topologia single-ended:


<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [22]:
# Sinal referência (campos)
t, ω_s, ω_lo, ϕ_s, ϕ_lo, Δω, Δϕ = sp.symbols('t, ω_s, ω_{LO}, ϕ_s, ϕ_{LO}, Δω, Δϕ', real=True)
P_s, P_lo, R = sp.symbols('P_s, P_{LO}, R_d', real=True, positive=True)

A_s = sp.Function('A', real=True)(t)
A_lo = sp.Function('A_{LO}', real=True)(t)
ϕ = sp.Function('ϕ', real=True)(t)

E_s  = A_s*sp.exp(j*(ω_s*t + ϕ_s))
E_lo = A_lo*sp.exp(j*(ω_lo*t + ϕ_lo))

# Topologia do Artigo -> 2 splitter, 1 defasador, 2 acopladores de 90°
Ei = Matrix([[ E_s ], 
             [ 0 ], 
             [ 0 ], 
             [ E_lo ]])

T1 = Matrix([[1, j, 0, 0],
             [j, 1, 0, 0],
             [0, 0, 1, j],
             [0, 0, j, 1]]) * (1/sp.sqrt(2)) * (sp.sqrt(2)) 

T2 = Matrix([[1, 0, 0, 0],
             [0, 0, 1, 0],
             [0, 1, 0, 0],
             [0, 0, 0, j]])

T3 = Matrix([[1, j, 0, 0],
             [j, 1, 0, 0],
             [0, 0, 1, j],
             [0, 0, j, 1]]) * (1/sp.sqrt(2))

print('Sinais de entrada da híbrida óptica:')
symdisp('E_i = ', Ei)

print('Função de transferência da híbrida óptica:')
symdisp('T = ', T3 * T2 * T1)

print('Modelagem:')
symdisp('T1 * E_i = ', T1 * Ei)
symdisp('T2 * T1 * E_i = ', T2 * T1 * Ei )
symdisp('T3 * T2 * T1 * E_i = ', T3 * T2 * T1 * Ei )

print('Sinais de saída da híbrida óptica / Sinais de entrada dos fotodiodos:')
Eo = T3 * T2 * T1 * Ei
symdisp('E_o = ', Eo)

#BPD
print('Saída com topologia balanceada:')
sI = bpd(Eo[1], Eo[0])
sQ = bpd(Eo[2], Eo[3])

symdisp('BPD | s_I = i_1 - i_2 = ', sp.re(sI).simplify().expand() )
symdisp('BPD | s_Q = i_4 - i_3 = ', sp.re(sQ).simplify().expand() )

#SER
print('Saída com topologia single-ended:')
sI = bpd(Eo[1], 0)
sQ = bpd(Eo[2], 0)

symdisp('SER | s_I = i_1 = ', sp.re(sI).simplify().expand() )
symdisp('SER | s_Q = i_3 = ', sp.re(sQ).simplify().expand() )

Sinais de entrada da híbrida óptica:


<IPython.core.display.Math object>

Função de transferência da híbrida óptica:


<IPython.core.display.Math object>

Modelagem:


...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Saída com topologia single-ended:


<IPython.core.display.Math object>

<IPython.core.display.Math object>